In [2]:
import os
import glob
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
import wandb
import matplotlib.pyplot as plt


In [3]:
wandb.login()

wandb: Currently logged in as: dwightsamu2 (dwightsamu2-ucreative) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

Configuración

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_dir_base = r"Datos\Augmented"
model_dir = "Modelos"
batch_size = 100
num_classes = 3


Tranformar datos

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


Modelos

In [ ]:
def get_model(model_name, num_classes):
    if model_name == "mobilenet":
        model = models.mobilenet_v2(pretrained=False)
        model.classifier[1] = nn.Linear(model.last_channel, num_classes)
    elif model_name == "vgg16":
        model = models.vgg16(pretrained=False)
        model.classifier[6] = nn.Linear(4096, num_classes)
    else:
        raise ValueError(f"Modelo no soportado: {model_name}")
    return model.to(device)

Metricas 

In [7]:
for model_path in glob.glob(os.path.join(model_dir, "*.pth")):
    filename = os.path.basename(model_path).replace(".pth", "")
    parts = filename.split("_")
    
    if len(parts) < 3:
        print(f"❌ Nombre de archivo inesperado: {filename}")
        continue

    model_name = parts[0]
    model_name = parts[0]
    conjunto = f"conjunto_{'_'.join(parts[2:])}" if parts[1] == "conjunto" else f"conjunto_{'_'.join(parts[1:])}"
    
    # Limpieza de ruta
    conjunto = os.path.normpath(conjunto)
    data_path = os.path.normpath(os.path.join(data_dir_base, conjunto))

    print(f"\n🔍 Evaluando: {model_name} con conjunto {conjunto}")
    print(f"📂 Buscando datos en: {data_path}")
    
    run = wandb.init(
        project="evaluacion-modelos",
        name=f"{model_name}_{conjunto}",
        reinit=True
    )

    # Verificación del directorio de datos
    if not os.path.isdir(data_path):
        print(f"⚠️ No existe el directorio de datos: {data_path}")
        run.finish()
        continue

    dataset = datasets.ImageFolder(data_path, transform=transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    # Cargar modelo
    try:
        model = get_model(model_name, num_classes)
        model.load_state_dict(torch.load(model_path, map_location=device))
        model.eval()
    except Exception as e:
        print(f"❌ Error al cargar modelo {filename}: {e}")
        run.finish()
        continue

    # Predicciones
    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    if not all_preds or not all_labels:
        print(f"⚠️ Sin predicciones para {model_name} - {conjunto}")
        run.finish()
        continue

    # Métricas
    acc = accuracy_score(all_labels, all_preds)
    prec = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    rec = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)

    wandb.log({
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1_score": f1
    })

    # Matriz de confusión
    cm = confusion_matrix(all_labels, all_preds)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=dataset.classes)
    fig = disp.plot(cmap='Blues').figure_
    wandb.log({"confusion_matrix": wandb.Image(fig)})
    plt.close(fig)

    print(f"✅ Resultados logueados: acc={acc:.3f}, f1={f1:.3f}")
    run.finish()



🔍 Evaluando: mobilenet con conjunto conjunto_bilateral
📂 Buscando datos en: Datos\Augmented\conjunto_bilateral


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


c:\Users\dwigh\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\dwigh\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


✅ Resultados logueados: acc=0.441, f1=0.204


accuracy,▁
f1_score,▁
precision,▁
recall,▁
accuracy,0.4409
f1_score,0.20409
precision,0.14726
recall,0.33233


...

accuracy,▁
f1_score,▁
precision,▁
recall,▁
accuracy,0.4761
f1_score,0.28404
precision,0.57988
recall,0.37381



🔍 Evaluando: vgg16 con conjunto conjunto_raw
📂 Buscando datos en: Datos\Augmented\conjunto_raw


c:\Users\dwigh\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\dwigh\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


✅ Resultados logueados: acc=0.575, f1=0.477


accuracy,▁
f1_score,▁
precision,▁
recall,▁
accuracy,0.57503
f1_score,0.47671
precision,0.80937
recall,0.49206
